In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import warnings
import pymc as pm
import arviz as az
import causalpy 
import seaborn as sns
from patsy import build_design_matrices, dmatrices
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

from library.synthetic_control import *
from library.data_generator import generate_gaussian_process_data
from library.synthetic_did import SyntheticDIDModel
from library.synthetic_bayes import WeightedSumFitter


plt.style.use('ggplot')
warnings.filterwarnings("ignore")

import os
import json
import numpy as np
import pandas as pd
import arviz as az
import pymc as pm
from patsy import dmatrices

In [2]:
real_data = pd.read_parquet('data/data.pt')

In [3]:
data_bayes = real_data.pivot(index='time', columns='shopno', values='preprocessed_avg_delivery').reset_index(drop=True).rename_axis(None, axis=1)

In [4]:
def bayesian_gp_synthetic_model(t, y, X, use_covariates=True):
    y = np.asarray(y).flatten()

    with pm.Model() as model:
        t_shared = pm.MutableData("t", t.reshape(-1, 1))

        if use_covariates:
            X_shared = pm.MutableData("X", X)
            beta_lin   = pm.Dirichlet("beta_lin", a=np.ones(X.shape[1]))
            intercept  = pm.Normal("intercept", mu=0, sigma=5)
            linear_term = intercept + pm.math.dot(X_shared, beta_lin)
        else:
            linear_term = 0

        sigma_f = pm.HalfNormal("sigma_f", sigma=2)
        ell     = pm.HalfNormal("ell", sigma=2)
        cov_func = sigma_f**2 * pm.gp.cov.ExpQuad(1, ell)
        gp = pm.gp.Latent(cov_func=cov_func)
        gp_effect = gp.prior("gp_effect", X=t_shared)

        mu = pm.Deterministic("mu", linear_term + gp_effect)
        sigma = pm.HalfNormal("sigma", sigma=2)

        y_shared = pm.MutableData("y", y)   
        pm.Normal("y_hat", mu=mu, sigma=sigma, observed=y_shared)

        idata = pm.sample(   draws=300,
                             tune=400,
                             chains=2,
                             return_inferencedata=True,
                             target_accept=0.95,
                             random_seed=42 )
        posterior_predictive = pm.sample_posterior_predictive(idata)

    return idata, posterior_predictive, model


In [5]:


def run_bayesian_synthetic_control_analysis(
    model_name: str,
    data: pd.DataFrame,
    output_dir: str = "results",
    metric: str = "summary",
    T0: int = 70,
    effect_sizes: list = None,
    hdi_prob: float = 0.97
) -> pd.DataFrame:

    if effect_sizes is None:
        effect_sizes = [0, 5, 10, 15, 20]

    units = data.columns.tolist()
    results = []

    fp_flags = []
    for response in units:
        treatment_time = T0
        predictors = [c for c in units if c != response]
        formula    = f"{response} ~ " + " + ".join(predictors)
        datapre = data_bayes[data_bayes.index < treatment_time]
        datapost = data_bayes[data_bayes.index >= treatment_time]
        y, X = dmatrices(formula, datapre)
        pre_y, pre_X = np.asarray(y), np.asarray(X)
        (new_y, new_x) = build_design_matrices(
            [y.design_info, X.design_info], datapost
        )
        post_X = np.asarray(new_x)
        post_y = np.asarray(new_y)
        del y, X
        y_pre_dm, X_pre_dm = dmatrices(formula, datapre)
        design_infos = [y_pre_dm.design_info, X_pre_dm.design_info]
        y_pre = np.asarray(y_pre_dm)   
        X_pre = np.asarray(X_pre_dm)  
        (new_y_dm, new_x_dm) = build_design_matrices(design_infos, datapost)
        y_post = np.asarray(new_y_dm)   
        X_post = np.asarray(new_x_dm)   
        y_pre_model = y_pre.flatten()  
        pre_t = np.arange(y_pre_model.shape[0]) 
        idata, posterior_predictive_pre, model = bayesian_gp_synthetic_model(
            pre_t, y_pre, X_pre, use_covariates=True
        )
        idata, posterior_predictive_pre, model = bayesian_gp_synthetic_model(pre_t, y_pre_model, X_pre, use_covariates=True)
        y_post_model = y_post.flatten()
        post_t = np.arange(pre_t[-1] + 1, pre_t[-1] + 1 + y_post_model.shape[0])
        with model:
            pm.set_data(new_data={"t": post_t.reshape(-1, 1), "X": X_post, "y": y_post_model})
            posterior_predictive_post = pm.sample_posterior_predictive(trace=idata, var_names=["y_hat"])
        post_posterior_mean = (
            posterior_predictive_post.posterior_predictive["y_hat"][:, :, :T0]
            .stack(samples=("chain", "draw"))
            .mean(axis=1)
        )
        actual_post = data_bayes.loc[datapost.index, f"{response}"].values
        att_time_series = actual_post - post_posterior_mean
        y_hat_pre = posterior_predictive_pre.posterior_predictive["y_hat"]
        y_hat_post = posterior_predictive_post.posterior_predictive["y_hat"]
        pre_posterior_mean = y_hat_pre.stack(samples=("chain", "draw")).mean(axis=1).values
        post_posterior_mean = y_hat_post.stack(samples=("chain", "draw")).mean(axis=1).values
        posterior_mean_full = np.concatenate([pre_posterior_mean, post_posterior_mean])
        att_mean = np.mean(att_time_series)
        actual_post = data_bayes.loc[datapost.index, f"{response}"].values
        att_time_series = actual_post - post_posterior_mean
        att_mean = np.mean(att_time_series)
        low, high = az.summary(posterior_predictive_post.posterior_predictive)[['hdi_3%', 'hdi_97%']].mean()
        fp_flags.append((low > 0) or (high < 0))
    type1_error = sum(fp_flags) / len(units)
    print(f"Type I error (false-positive rate): {type1_error:.3f}\n")

    for eff in effect_sizes[1:]:
        fn_flags = []
        ests = []
        for response in units:

            treatment_time = T0
            predictors = [c for c in units if c != response]
            formula    = f"{response} ~ " + " + ".join(predictors)
            datapre = data_bayes[data_bayes.index < treatment_time]
            datapost = data_bayes[data_bayes.index >= treatment_time]
            y, X = dmatrices(formula, datapre)
            pre_y, pre_X = np.asarray(y), np.asarray(X)
            (new_y, new_x) = build_design_matrices(
                [y.design_info, X.design_info], datapost
            )
            post_X = np.asarray(new_x)
            post_y = np.asarray(new_y)
            del y, X
            y_pre_dm, X_pre_dm = dmatrices(formula, datapre)
            design_infos = [y_pre_dm.design_info, X_pre_dm.design_info]
            y_pre = np.asarray(y_pre_dm)   
            X_pre = np.asarray(X_pre_dm)  
            (new_y_dm, new_x_dm) = build_design_matrices(design_infos, datapost)
            y_post = np.asarray(new_y_dm)   
            X_post = np.asarray(new_x_dm)   
            y_pre_model = y_pre.flatten()  
            pre_t = np.arange(y_pre_model.shape[0]) 
            idata, posterior_predictive_pre, model = bayesian_gp_synthetic_model(
                pre_t, y_pre, X_pre, use_covariates=True
            )
            idata, posterior_predictive_pre, model = bayesian_gp_synthetic_model(pre_t, y_pre_model, X_pre, use_covariates=True)
            y_post_model = y_post.flatten()
            post_t = np.arange(pre_t[-1] + 1, pre_t[-1] + 1 + y_post_model.shape[0])
            with model:
                pm.set_data(new_data={"t": post_t.reshape(-1, 1), "X": X_post, "y": y_post_model})
                posterior_predictive_post = pm.sample_posterior_predictive(trace=idata, var_names=["y_hat"])
            post_posterior_mean = (
                posterior_predictive_post.posterior_predictive["y_hat"][:, :, :T0]
                .stack(samples=("chain", "draw"))
                .mean(axis=1)
            )
            actual_post = data_bayes.loc[datapost.index, f"{response}"].values
            att_time_series = actual_post - post_posterior_mean
            y_hat_pre = posterior_predictive_pre.posterior_predictive["y_hat"]
            y_hat_post = posterior_predictive_post.posterior_predictive["y_hat"]
            pre_posterior_mean = y_hat_pre.stack(samples=("chain", "draw")).mean(axis=1).values
            post_posterior_mean = y_hat_post.stack(samples=("chain", "draw")).mean(axis=1).values
            posterior_mean_full = np.concatenate([pre_posterior_mean, post_posterior_mean])
            att_mean = np.mean(att_time_series)
            actual_post = data_bayes.loc[datapost.index, f"{response}"].values
            att_time_series = actual_post - post_posterior_mean
            att_mean = np.mean(att_time_series)
            low, high = az.summary(posterior_predictive_post.posterior_predictive)[['hdi_3%', 'hdi_97%']].mean()

            mean_eff = 1

            detected = (low > 0) or (high < 0)
            fn_flags.append(not detected)
            ests.append(mean_eff)
        type2_error = sum(fn_flags) / len(units)
        errors = np.array(ests) - eff
        rmspe = np.sqrt(np.mean(errors ** 2))
        print(f"=== Effect {eff}% ===")
        print(f"Type II error (false-negative rate): {type2_error:.3f}")
        print(f"RMSPE of effect estimates: {rmspe:.3f}\n")
        results.append({
            "effect_size": eff,
            "type1_error": type1_error,
            "type2_error": type2_error,
            "rmspe": rmspe,
        })

    summary = pd.DataFrame(results)
    print("=== Summary ===")
    print(summary.to_string(index=False))

    result_dict = {
        "model": model_name,
        "metric": metric,
        "summary": summary.to_dict(orient="records")
    }
    filename = os.path.join(f"summary_{model_name}_{metric}.json")
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(result_dict, f, ensure_ascii=False, indent=2)
    print(f"Results saved to {filename}")

    return summary


In [1]:
data_bayes = real_data.pivot(index='time', columns='shopno', values='preprocessed_avg_delivery').reset_index(drop=True).rename_axis(None, axis=1)

run_bayesian_synthetic_control_analysis(
    model_name="BayesianSyntheticControlGP",
    data=data_bayes,
    metric="preprocessed_avg_delivery",
    T0=70
)

In [ ]:
data_bayes = real_data.pivot(index='time', columns='shopno', values='preprocessed_orders_per_courier').reset_index(drop=True).rename_axis(None, axis=1)

run_bayesian_synthetic_control_analysis(
    model_name="BayesianSyntheticControlGP",
    data=data_bayes,
    metric="preprocessed_orders_per_courier",
    T0=70
)

In [ ]:
data_bayes = real_data.pivot(index='time', columns='shopno', values='preprocessed_distance').reset_index(drop=True).rename_axis(None, axis=1)

run_bayesian_synthetic_control_analysis(
    model_name="BayesianSyntheticControlGP",
    data=data_bayes,
    metric="preprocessed_distance",
    T0=70
)

In [ ]:
data_bayes = real_data.pivot(index='time', columns='shopno', values='preprocessed_avg_collection_time').reset_index(drop=True).rename_axis(None, axis=1)

run_bayesian_synthetic_control_analysis(
    model_name="BayesianSyntheticControlGP",
    data=data_bayes,
    metric="preprocessed_avg_collection_time",
    T0=70
)

In [ ]:
data_bayes = real_data.pivot(index='time', columns='shopno', values='preprocessed_percent_late').reset_index(drop=True).rename_axis(None, axis=1)

run_bayesian_synthetic_control_analysis(
    model_name="BayesianSyntheticControlGP",
    data=data_bayes,
    metric="preprocessed_percent_late",
    T0=70
)